In [ ]:
#Exploring the concept of taking two data frames from two seperate queries and applying RITA calculations
## C2 Beacons found at 192.168.120.158 on port 443 communicating with 192.168.128.50


In [29]:
#Elasticsearch connector
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import eland as ed
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [30]:
es = Elasticsearch(['https://192.168.1.181:9200'], timeout=30, max_retries=10, retry_on_timeout=True,                  
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))

# defining the search statement to get all records in an index
searchContext = Search(using=es, index='*:so-*', doc_type='doc')
es.info()

ObjectApiResponse({'name': 'voodoo-onion', 'cluster_name': 'voodoo-onion', 'cluster_uuid': 'dP1VycCSTQ2EPeCVxQ25GA', 'version': {'number': '7.16.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb', 'build_date': '2021-12-18T19:42:46.604893745Z', 'build_snapshot': False, 'lucene_version': '8.10.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [31]:
query_since_ts = 0

#body_filter = '{"query":{"bool":{"filter":[{"range":{@timestamp":{"gt":'+ str(query_since_ts) + '}}}]}},"sort":[{"@timstamp":{"order":"asc"}}]}'

body_filter = '{"query":{"@timestamp":{}},"size=10000":[{"@timestamp":{"order":"asc"}}]}'

In [32]:
#Firewall 192.168.1.154

s1 = searchContext.query('query_string', query='(event.category:(network OR network_traffic) AND network.transport:tcp) AND destination.port:(53 OR 80 OR 8080 OR 443) AND (@timestamp > "now-3d/d") AND NOT source.ip:192.168.1.154')
s2 = searchContext.query('query_string', query='event.module:zeek')

In [33]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)


In [34]:
#data frame 1 for s1 which is search query 1
response = s1.execute()
df1 = pd.DataFrame(columns=['ts','source.ip','destination.ip','destination.port'])
if response.success():
    for d in s1[:10000]:
        try:
            df1 = df1.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'destination.port': d['destination']['port']}, ignore_index=True)
        except KeyError:
            pass       

display(df)

df1.info()

,ts,source.ip,destination.ip,destination.port
0,2021-12-29T22:20:52.526Z,192.168.128.15,23.66.101.177,80
1,2021-12-29T22:20:53.091Z,192.168.128.15,23.66.101.177,80
2,2021-12-29T22:20:52.576Z,192.168.128.15,23.66.101.177,80
3,2021-12-29T22:20:54.325Z,192.168.128.15,23.66.101.177,80
4,2021-12-29T22:20:52.593Z,192.168.128.15,23.66.101.177,80
...,...,...,...,...
3166,2022-01-01T07:36:44.472Z,192.168.128.26,168.63.250.82,80
3167,2022-01-01T07:36:44.171Z,192.168.128.26,168.63.250.82,80
3168,2022-01-01T07:36:43.891Z,192.168.128.26,168.63.250.82,80
3169,2022-01-01T07:36:43.584Z,192.168.128.26,168.63.250.82,80


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3186 entries, 0 to 3185
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                3186 non-null   object
 1   source.ip         3186 non-null   object
 2   destination.ip    3186 non-null   object
 3   destination.port  3186 non-null   object
dtypes: object(4)
memory usage: 99.7+ KB


In [35]:
#data frame 2 for s2 which is search query 2
response = s2.execute()
df2 = pd.DataFrame(columns=['ts','source.ip','destination.ip','destination.port'])
if response.success():
    for d in s2[:10000]:
        try:
            df2 = df2.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'destination.port': d['destination']['port']}, ignore_index=True)
        except KeyError:
            pass       

display(df)

df2.info()

,ts,source.ip,destination.ip,destination.port
0,2021-12-29T22:20:52.526Z,192.168.128.15,23.66.101.177,80
1,2021-12-29T22:20:53.091Z,192.168.128.15,23.66.101.177,80
2,2021-12-29T22:20:52.576Z,192.168.128.15,23.66.101.177,80
3,2021-12-29T22:20:54.325Z,192.168.128.15,23.66.101.177,80
4,2021-12-29T22:20:52.593Z,192.168.128.15,23.66.101.177,80
...,...,...,...,...
3166,2022-01-01T07:36:44.472Z,192.168.128.26,168.63.250.82,80
3167,2022-01-01T07:36:44.171Z,192.168.128.26,168.63.250.82,80
3168,2022-01-01T07:36:43.891Z,192.168.128.26,168.63.250.82,80
3169,2022-01-01T07:36:43.584Z,192.168.128.26,168.63.250.82,80


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9569 entries, 0 to 9568
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                9569 non-null   object
 1   source.ip         9569 non-null   object
 2   destination.ip    9569 non-null   object
 3   destination.port  9569 non-null   object
dtypes: object(4)
memory usage: 299.2+ KB


In [36]:
# assign field/column names to variables
f_timestamp = 'ts'
f_src_ip = 'source.ip'
f_dst_ip = 'destination.ip'
f_dst_port = 'destination.port'
f_delimiter = '\t'


In [37]:
df1['ts'] = pd.to_datetime(df1['ts'],infer_datetime_format=True)
df1.head(10)

,ts,source.ip,destination.ip,destination.port
0,2021-12-29 22:20:52.526000+00:00,192.168.128.15,23.66.101.177,80
1,2021-12-29 22:20:53.091000+00:00,192.168.128.15,23.66.101.177,80
2,2021-12-29 22:20:52.576000+00:00,192.168.128.15,23.66.101.177,80
3,2021-12-29 22:20:54.325000+00:00,192.168.128.15,23.66.101.177,80
4,2021-12-29 22:20:52.593000+00:00,192.168.128.15,23.66.101.177,80
5,2021-12-29 22:20:53.125000+00:00,192.168.128.15,23.66.101.177,80
6,2021-12-29 22:20:55.257000+00:00,192.168.128.15,23.66.101.177,80
7,2021-12-29 22:20:53.427000+00:00,192.168.128.15,23.66.101.177,80
8,2021-12-29 22:20:53.405000+00:00,192.168.128.15,23.66.101.177,80
9,2021-12-29 22:20:55.175000+00:00,192.168.128.15,23.66.101.177,80


In [38]:
df2['ts'] = pd.to_datetime(df2['ts'],infer_datetime_format=True)
df2.head(10)

,ts,source.ip,destination.ip,destination.port
0,2021-12-29 23:59:39.018000+00:00,192.168.1.220,224.0.0.252,5355
1,2021-12-29 23:59:56.601000+00:00,192.168.1.154,216.239.34.10,53
2,2021-12-29 23:58:19.806000+00:00,192.168.1.220,192.168.1.255,137
3,2021-12-29 23:58:23.838000+00:00,192.168.1.220,192.168.1.255,137
4,2021-12-29 23:57:34.767000+00:00,192.168.1.220,192.168.1.255,137
5,2021-12-29 23:58:19.040000+00:00,192.168.1.220,192.168.1.255,137
6,2021-12-29 23:59:01.532000+00:00,192.168.1.220,224.0.0.251,5353
7,2021-12-29 23:58:14.243000+00:00,192.168.1.220,192.168.1.255,137
8,2021-12-29 23:57:38.768000+00:00,192.168.1.220,192.168.1.255,137
9,2021-12-29 23:58:17.401000+00:00,192.168.1.220,192.168.1.255,137


In [39]:
# Merge the two to create our final dataframe
df = pd.concat([df1, df2], ignore_index=True)
print(df.head(5))
print(df.tail(5))

                                ts       source.ip destination.ip  \
0 2021-12-29 22:20:52.526000+00:00  192.168.128.15  23.66.101.177   
1 2021-12-29 22:20:53.091000+00:00  192.168.128.15  23.66.101.177   
2 2021-12-29 22:20:52.576000+00:00  192.168.128.15  23.66.101.177   
3 2021-12-29 22:20:54.325000+00:00  192.168.128.15  23.66.101.177   
4 2021-12-29 22:20:52.593000+00:00  192.168.128.15  23.66.101.177   

  destination.port  
0               80  
1               80  
2               80  
3               80  
4               80  
                                    ts      source.ip destination.ip  \
12750 2021-12-29 21:12:11.575000+00:00  192.168.1.220  192.168.1.255   
12751 2021-12-29 21:11:57.461000+00:00  192.168.1.220  192.168.1.255   
12752 2021-12-29 21:12:44.810000+00:00  192.168.1.220  192.168.1.255   
12753 2021-12-29 21:13:05.585000+00:00  192.168.1.220  192.168.1.255   
12754 2021-12-29 21:12:13.974000+00:00  192.168.1.220  192.168.1.255   

      destination.port  
1

In [40]:

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_port,]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_port,]
# columns to display after the analysis
columns_to_display = ['tsScore','conn_count',f_src_ip,f_dst_ip,f_dst_port,'deltas']

In [41]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
df = df.groupby(columns_to_groupby).agg(list)
df.head(50)

ts
source.ip     destination.ip  destination.port                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
127.0.0.1     127.0.0.1       4506                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             [2021-12-29 20:38:42.772000+00:00, 2021-12-29 20:38:42.810000+00:00, 2021-12-29 20:38:42.772000+00:00, 2021-12-29 20:38:42.811000+00:00]
                              47660                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  [2021-12-29 20:38:42.776000+00:00]
                              47761                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  [2021-12-29 20:38:42.805000+00:00]
                              47762                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  [2021-12-29 20:38:42.805000+00:00]
                              47763                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  [2021-12-29 20:38:42.805000+00:00]
192.168.1.154 104.208.16.90   443                                                                            

In [42]:
df.reset_index(inplace=True)
df.head(5)

,source.ip,destination.ip,destination.port,ts
0,127.0.0.1,127.0.0.1,4506,"[2021-12-29 20:38:42.772000+00:00, 2021-12-29 20:38:42.810000+00:00, 2021-12-29 20:38:42.772000+00:00, 2021-12-29 20:38:42.811000+00:00]"
1,127.0.0.1,127.0.0.1,47660,[2021-12-29 20:38:42.776000+00:00]
2,127.0.0.1,127.0.0.1,47761,[2021-12-29 20:38:42.805000+00:00]
3,127.0.0.1,127.0.0.1,47762,[2021-12-29 20:38:42.805000+00:00]
4,127.0.0.1,127.0.0.1,47763,[2021-12-29 20:38:42.805000+00:00]


In [43]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
df['conn_count'] = df[f_timestamp].apply(lambda x: len(x))
df.head(2)

,source.ip,destination.ip,destination.port,ts,conn_count
0,127.0.0.1,127.0.0.1,4506,"[2021-12-29 20:38:42.772000+00:00, 2021-12-29 20:38:42.810000+00:00, 2021-12-29 20:38:42.772000+00:00, 2021-12-29 20:38:42.811000+00:00]",4
1,127.0.0.1,127.0.0.1,47660,[2021-12-29 20:38:42.776000+00:00],1


In [44]:
#Filter out traffic where the connection count is quite small
df = df.loc[df['conn_count'] > 50]
df.shape

(19, 5)

In [45]:
df[f_timestamp] = df[f_timestamp].apply(lambda x: sorted(x))
df.head()

,source.ip,destination.ip,destination.port,ts,conn_count
40,192.168.1.154,192.168.1.181,5044,"[2021-12-29 19:59:57.191000+00:00, 2021-12-29 20:00:12.223000+00:00, 2021-12-29 20:00:46.944000+00:00, 2021-12-29 20:01:03.275000+00:00, 2021-12-29 20:01:12.297000+00:00, 2021-12-29 20:01:49.339000+00:00, 2021-12-29 20:01:49.987000+00:00, 2021-12-29 20:02:11.571000+00:00, 2021-12-29 20:02:20.583000+00:00, 2021-12-29 20:02:49.461000+00:00, 2021-12-29 20:03:11.345000+00:00, 2021-12-29 20:03:28.643000+00:00, 2021-12-29 20:03:31.678000+00:00, 2021-12-29 20:04:13.333000+00:00, 2021-12-29 20:04:25.162000+00:00, 2021-12-29 20:04:29.780000+00:00, 2021-12-29 20:04:34.355000+00:00, 2021-12-29 20:05:...",305
96,192.168.1.154,40.90.4.205,53,"[2021-12-29 20:00:29.783000+00:00, 2021-12-29 20:00:57.088000+00:00, 2021-12-29 20:01:29.801000+00:00, 2021-12-29 20:04:07.536000+00:00, 2021-12-29 20:05:24.994000+00:00, 2021-12-29 20:08:29.789000+00:00, 2021-12-29 20:08:56.737000+00:00, 2021-12-29 20:08:56.737000+00:00, 2021-12-29 20:09:29.788000+00:00, 2021-12-29 20:12:07.543000+00:00, 2021-12-29 20:18:14.797000+00:00, 2021-12-29 20:21:52.548000+00:00, 2021-12-29 20:21:59.629000+00:00, 2021-12-29 20:22:29.806000+00:00, 2021-12-29 20:24:44.797000+00:00, 2021-12-29 20:27:24.759000+00:00, 2021-12-29 20:28:07.566000+00:00, 2021-12-29 20:29:...",64
153,192.168.1.202,224.0.0.251,5353,"[2021-12-29 19:59:51.255000+00:00, 2021-12-29 19:59:56.163000+00:00, 2021-12-29 19:59:57.100000+00:00, 2021-12-29 19:59:59.576000+00:00, 2021-12-29 20:00:08.718000+00:00, 2021-12-29 20:00:10.288000+00:00, 2021-12-29 20:00:14.983000+00:00, 2021-12-29 20:00:29.024000+00:00, 2021-12-29 20:00:31.547000+00:00, 2021-12-29 20:00:40.987000+00:00, 2021-12-29 20:00:48.111000+00:00, 2021-12-29 20:00:50.501000+00:00, 2021-12-29 20:00:51.447000+00:00, 2021-12-29 20:01:22.758000+00:00, 2021-12-29 20:01:33.271000+00:00, 2021-12-29 20:01:34.793000+00:00, 2021-12-29 20:01:36.967000+00:00, 2021-12-29 20:01:...",587
156,192.168.1.220,192.168.1.255,137,"[2021-12-29 19:59:44.044000+00:00, 2021-12-29 19:59:45.420000+00:00, 2021-12-29 19:59:46.295000+00:00, 2021-12-29 19:59:47.807000+00:00, 2021-12-29 19:59:49.429000+00:00, 2021-12-29 19:59:50.449000+00:00, 2021-12-29 19:59:57.402000+00:00, 2021-12-29 19:59:57.402000+00:00, 2021-12-29 20:00:05.235000+00:00, 2021-12-29 20:00:05.988000+00:00, 2021-12-29 20:00:07.625000+00:00, 2021-12-29 20:00:07.626000+00:00, 2021-12-29 20:00:08.387000+00:00, 2021-12-29 20:00:10.001000+00:00, 2021-12-29 20:00:14.784000+00:00, 2021-12-29 20:00:15.549000+00:00, 2021-12-29 20:00:16.320000+00:00, 2021-12-29 20:00:...",3501
158,192.168.1.220,224.0.0.251,5353,"[2021-12-29 19:59:44.044000+00:00, 2021-12-29 19:59:53.463000+00:00, 2021-12-29 19:59:54.488000+00:00, 2021-12-29 20:00:07.627000+00:00, 2021-12-29 20:00:15.797000+00:00, 2021-12-29 20:00:17.211000+00:00, 2021-12-29 20:01:22.680000+00:00, 2021-12-29 20:01:29.632000+00:00, 2021-12-29 20:01:33.037000+00:00, 2021-12-29 20:01:40.220000+00:00, 2021-12-29 20:01:42.608000+00:00, 2021-12-29 20:01:50.953000+00:00, 2021-12-29 20:01:53.352000+00:00, 2021-12-29 20:01:56.766000+00:00, 2021-12-29 20:02:17.133000+00:00, 2021-12-29 20:02:18.137000+00:00, 2021-12-29 20:02:25.323000+00:00, 2021-12-29 20:02:...",1193


In [46]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
df['deltas'] = df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
df.head(2)

,source.ip,destination.ip,destination.port,ts,conn_count,deltas
40,192.168.1.154,192.168.1.181,5044,"[2021-12-29 19:59:57.191000+00:00, 2021-12-29 20:00:12.223000+00:00, 2021-12-29 20:00:46.944000+00:00, 2021-12-29 20:01:03.275000+00:00, 2021-12-29 20:01:12.297000+00:00, 2021-12-29 20:01:49.339000+00:00, 2021-12-29 20:01:49.987000+00:00, 2021-12-29 20:02:11.571000+00:00, 2021-12-29 20:02:20.583000+00:00, 2021-12-29 20:02:49.461000+00:00, 2021-12-29 20:03:11.345000+00:00, 2021-12-29 20:03:28.643000+00:00, 2021-12-29 20:03:31.678000+00:00, 2021-12-29 20:04:13.333000+00:00, 2021-12-29 20:04:25.162000+00:00, 2021-12-29 20:04:29.780000+00:00, 2021-12-29 20:04:34.355000+00:00, 2021-12-29 20:05:...",305,"[15.0, 34.0, 16.0, 9.0, 37.0, 0.0, 21.0, 9.0, 28.0, 21.0, 17.0, 3.0, 41.0, 11.0, 4.0, 4.0, 62.0, 14.0, 29.0, 7.0, 21.0, 0.0, 8.0, 6.0, 1.0, 34.0, 44.0, 7.0, 21.0, 5.0, 8.0, 14.0, 35.0, 8.0, 20.0, 136.0, 27.0, 15.0, 293.0, 16.0, 38.0, 5.0, 25.0, 41.0, 15.0, 1.0, 1.0, 35.0, 13.0, 1.0, 82.0, 15.0, 20.0, 8.0, 5.0, 7.0, 30.0, 13.0, 32.0, 7.0, 27.0, 4.0, 40.0, 15.0, 11.0, 7.0, 25.0, 24.0, 27.0, 5.0, 20.0, 3.0, 5.0, 14.0, 8.0, 12.0, 31.0, 21.0, 7.0, 8.0, 12.0, 40.0, 6.0, 57.0, 8.0, 7.0, 26.0, 37.0, 13.0, 1.0, 5.0, 1.0, 8.0, 28.0, 2.0, 4.0, 2.0, 5.0, 2.0, 6.0, ...]"
96,192.168.1.154,40.90.4.205,53,"[2021-12-29 20:00:29.783000+00:00, 2021-12-29 20:00:57.088000+00:00, 2021-12-29 20:01:29.801000+00:00, 2021-12-29 20:04:07.536000+00:00, 2021-12-29 20:05:24.994000+00:00, 2021-12-29 20:08:29.789000+00:00, 2021-12-29 20:08:56.737000+00:00, 2021-12-29 20:08:56.737000+00:00, 2021-12-29 20:09:29.788000+00:00, 2021-12-29 20:12:07.543000+00:00, 2021-12-29 20:18:14.797000+00:00, 2021-12-29 20:21:52.548000+00:00, 2021-12-29 20:21:59.629000+00:00, 2021-12-29 20:22:29.806000+00:00, 2021-12-29 20:24:44.797000+00:00, 2021-12-29 20:27:24.759000+00:00, 2021-12-29 20:28:07.566000+00:00, 2021-12-29 20:29:...",64,"[27.0, 32.0, 157.0, 77.0, 184.0, 26.0, 0.0, 33.0, 157.0, 367.0, 217.0, 7.0, 30.0, 134.0, 159.0, 42.0, 92.0, 5.0, 0.0, 89.0, 195.0, 67.0, 32.0, 24.0, 0.0, 85.0, 225.0, 89.0, 75.0, 329.0, 529.0, 52.0, 40.0, 0.0, 50.0, 99.0, 221.0, 213.0, 0.0, 52.0, 0.0, 300.0, 300.0, 1042.0, 34.0, 100.0, 47.0, 590.0, 8.0, 51.0, 0.0, 63.0, 1720.0, 1.0, 0.0, 20.0, 1135.0, 1209.0, 0.0, 957.0, 1145.0, 4.0, 14.0]"


In [51]:
df['tsLow'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
df['tsMid'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
df['tsHigh'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
df['tsBowleyNum'] = df['tsLow'] + df['tsHigh'] - 2*df['tsMid']
df['tsBowleyDen'] = df['tsHigh'] - df['tsLow']
df['tsSkew'] = df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
df['tsMadm'] = df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
df['tsConnDiv'] = df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 60)

In [52]:
df.head(3)

,source.ip,destination.ip,destination.port,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv,tsSkewScore,tsMadmScore,tsConnCountScore,tsScore
0,192.168.128.15,23.66.101.177,80,"[2021-12-29 22:20:52.436000+00:00, 2021-12-29 22:20:52.447000+00:00, 2021-12-29 22:20:52.464000+00:00, 2021-12-29 22:20:52.464000+00:00, 2021-12-29 22:20:52.498000+00:00, 2021-12-29 22:20:52.498000+00:00, 2021-12-29 22:20:52.503000+00:00, 2021-12-29 22:20:52.516000+00:00, 2021-12-29 22:20:52.526000+00:00, 2021-12-29 22:20:52.552000+00:00, 2021-12-29 22:20:52.576000+00:00, 2021-12-29 22:20:52.576000+00:00, 2021-12-29 22:20:52.588000+00:00, 2021-12-29 22:20:52.593000+00:00, 2021-12-29 22:20:52.604000+00:00, 2021-12-29 22:20:52.621000+00:00, 2021-12-29 22:20:52.636000+00:00, 2021-12-29 22:20:...",197,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050000,1.0,1.000000,1.0,1.000000
1,192.168.128.50,8.240.196.126,80,"[2021-12-29 22:03:21.207000+00:00, 2021-12-29 22:03:21.246000+00:00, 2021-12-29 22:03:21.305000+00:00, 2021-12-29 22:03:21.331000+00:00, 2021-12-29 22:03:21.405000+00:00, 2021-12-29 22:03:21.444000+00:00, 2021-12-29 22:03:21.486000+00:00, 2021-12-29 22:03:21.523000+00:00, 2021-12-29 22:03:21.534000+00:00, 2021-12-29 22:03:21.588000+00:00, 2021-12-29 22:03:21.614000+00:00, 2021-12-29 22:03:21.670000+00:00, 2021-12-29 22:03:21.697000+00:00, 2021-12-29 22:03:21.750000+00:00, 2021-12-29 22:03:21.812000+00:00, 2021-12-29 22:03:21.817000+00:00, 2021-12-29 22:03:21.890000+00:00, 2021-12-29 22:03:...",159,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.150000,1.0,1.000000,1.0,1.000000
2,192.168.1.220,192.168.1.255,137,"[2021-12-29 19:59:44.044000+00:00, 2021-12-29 19:59:45.420000+00:00, 2021-12-29 19:59:46.295000+00:00, 2021-12-29 19:59:47.807000+00:00, 2021-12-29 19:59:49.429000+00:00, 2021-12-29 19:59:50.449000+00:00, 2021-12-29 19:59:57.402000+00:00, 2021-12-29 19:59:57.402000+00:00, 2021-12-29 20:00:05.235000+00:00, 2021-12-29 20:00:05.988000+00:00, 2021-12-29 20:00:07.625000+00:00, 2021-12-29 20:00:07.626000+00:00, 2021-12-29 20:00:08.387000+00:00, 2021-12-29 20:00:10.001000+00:00, 2021-12-29 20:00:14.784000+00:00, 2021-12-29 20:00:15.549000+00:00, 2021-12-29 20:00:16.320000+00:00, 2021-12-29 20:00:...",3501,"[1.0, 0.0, 1.0, 1.0, 1.0, 6.0, 0.0, 7.0, 0.0, 1.0, 0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0, 2.0, 11.0, 3.0, 0.0, 0.0, 5.0, 6.0, 34.0, 0.0, 9.0, 2.0, 0.0, 1.0, 5.0, 2.0, 5.0, 0.0, 4.0, 4.0, 2.0, 3.0, 12.0, 1.0, 1.0, 0.0, 2.0, 0.0, 2.0, 40.0, 6.0, 0.0, 15.0, 5.0, 3.0, 0.0, 1.0, 2.0, 0.0, 3.0, 1.0, 1.0, 0.0, 11.0, 4.0, 1.0, 0.0, 12.0, 37.0, 0.0, 1.0, 1.0, 2.0, 0.0, 1.0, 3.0, 3.0, 5.0, 1.0, 3.0, 2.0, 4.0, 1.0, 1.0, 6.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 5.0, 1.0, 0.0, 6.0, 2.0, 33.0, 8.0, 0.0, 7.0, 0.0, 0.0, ...]",0.0,1.0,4.0,2.0,4.0,0.5,1.0,240.133333,0.5,0.966667,1.0,0.822222


In [53]:
df['tsSkewScore'] = 1.0 - abs(df['tsSkew'])
df['tsMadmScore'] = 1.0 - df['tsMadm']/30.0
df['tsMadmScore'] = df['tsMadmScore'].apply(lambda x: 0 if x < 0 else x)
df['tsConnCountScore'] = (df['conn_count']) / df['tsConnDiv']
df['tsConnCountScore'] = df['tsConnCountScore'].apply(lambda x: 1.0 if x > 1.0 else x)
df['tsScore'] = (((df['tsSkewScore'] + df['tsMadmScore'] + df['tsConnCountScore']) / 3.0) * 1000) / 1000
df.sort_values(by= 'tsScore', ascending=False, inplace=True, ignore_index=True)
df[columns_to_display].head(30)

,tsScore,conn_count,source.ip,destination.ip,destination.port,deltas
0,1.000000,197,192.168.128.15,23.66.101.177,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
1,1.000000,159,192.168.128.50,8.240.196.126,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
2,0.893256,98,192.168.128.50,192.168.128.25,53,"[0.0, 0.0, 0.0, 97.0, 209.0, 1035.0, 36.0, 98.0, 29.0, 74.0, 300.0, 60.0, 120.0, 119.0, 240.0, 284.0, 120.0, 450.0, 0.0, 270.0, 52.0, 0.0, 0.0, 0.0, 0.0, 96.0, 299.0, 0.0, 135.0, 449.0, 135.0, 135.0, 1110.0, 0.0, 759.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 12.0, 29.0, 2.0, 1.0, 27.0, 48.0, 15.0, 0.0, 0.0, 0.0, 162.0, 0.0, 600.0, 0.0, 606.0, 0.0, 0.0, 0.0, 345.0, 0.0, 0.0, 20.0, 39.0]"
3,0.876315,93,192.168.128.50,192.168.120.158,443,"[120.0, 120.0, 120.0, 0.0, 0.0, 120.0, 0.0, 120.0, 0.0, 120.0, 461.0, 20.0, 120.0, 120.0, 120.0, 0.0, 0.0, 120.0, 120.0, 0.0, 120.0, 120.0, 0.0, 0.0, 120.0, 120.0, 0.0, 0.0, 120.0, 120.0, 120.0, 120.0, 120.0, 0.0, 120.0, 120.0, 120.0, 120.0, 0.0, 120.0, 120.0, 0.0, 120.0, 120.0, 0.0, 120.0, 0.0, 120.0, 120.0, 0.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 0.0, 120.0, 0.0, 120.0, 120.0, 0.0, 676.0, 0.0, 20.0, 120.0, 120.0, 120.0, 120.0, 120.0, 100.0, 20.0, 0.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 0.0, 120.0, 120.0, 120.0, 120.0]"
4,0.826746,97,192.168.128.50,192.168.120.158,443,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 696.0, 120.0, 120.0, 120.0, 120.0, 120.0, ..."
5,0.822222,3501,192.168.1.220,192.168.1.255,137,"[1.0, 0.0, 1.0, 1.0, 1.0, 6.0, 0.0, 7.0, 0.0, 1.0, 0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0, 2.0, 11.0, 3.0, 0.0, 0.0, 5.0, 6.0, 34.0, 0.0, 9.0, 2.0, 0.0, 1.0, 5.0, 2.0, 5.0, 0.0, 4.0, 4.0, 2.0, 3.0, 12.0, 1.0, 1.0, 0.0, 2.0, 0.0, 2.0, 40.0, 6.0, 0.0, 15.0, 5.0, 3.0, 0.0, 1.0, 2.0, 0.0, 3.0, 1.0, 1.0, 0.0, 11.0, 4.0, 1.0, 0.0, 12.0, 37.0, 0.0, 1.0, 1.0, 2.0, 0.0, 1.0, 3.0, 3.0, 5.0, 1.0, 3.0, 2.0, 4.0, 1.0, 1.0, 6.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 5.0, 1.0, 0.0, 6.0, 2.0, 33.0, 8.0, 0.0, 7.0, 0.0, 0.0, ...]"
6,0.812698,1193,192.168.1.220,224.0.0.251,5353,"[9.0, 1.0, 13.0, 8.0, 1.0, 65.0, 6.0, 3.0, 7.0, 2.0, 8.0, 2.0, 3.0, 20.0, 1.0, 7.0, 14.0, 30.0, 5.0, 12.0, 1.0, 1.0, 10.0, 6.0, 2.0, 1.0, 18.0, 2.0, 11.0, 32.0, 8.0, 1.0, 2.0, 4.0, 1.0, 1.0, 1.0, 14.0, 17.0, 9.0, 51.0, 2.0, 11.0, 6.0, 3.0, 3.0, 14.0, 1.0, 3.0, 2.0, 1.0, 1.0, 2.0, 8.0, 1.0, 1.0, 1.0, 47.0, 1.0, 3.0,

In [54]:
df.loc[df['tsScore'] > 0.80, columns_to_display]

,tsScore,conn_count,source.ip,destination.ip,destination.port,deltas
0,1.000000,197,192.168.128.15,23.66.101.177,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
1,1.000000,159,192.168.128.50,8.240.196.126,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
2,0.893256,98,192.168.128.50,192.168.128.25,53,"[0.0, 0.0, 0.0, 97.0, 209.0, 1035.0, 36.0, 98.0, 29.0, 74.0, 300.0, 60.0, 120.0, 119.0, 240.0, 284.0, 120.0, 450.0, 0.0, 270.0, 52.0, 0.0, 0.0, 0.0, 0.0, 96.0, 299.0, 0.0, 135.0, 449.0, 135.0, 135.0, 1110.0, 0.0, 759.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 12.0, 29.0, 2.0, 1.0, 27.0, 48.0, 15.0, 0.0, 0.0, 0.0, 162.0, 0.0, 600.0, 0.0, 606.0, 0.0, 0.0, 0.0, 345.0, 0.0, 0.0, 20.0, 39.0]"
3,0.876315,93,192.168.128.50,192.168.120.158,443,"[120.0, 120.0, 120.0, 0.0, 0.0, 120.0, 0.0, 120.0, 0.0, 120.0, 461.0, 20.0, 120.0, 120.0, 120.0, 0.0, 0.0, 120.0, 120.0, 0.0, 120.0, 120.0, 0.0, 0.0, 120.0, 120.0, 0.0, 0.0, 120.0, 120.0, 120.0, 120.0, 120.0, 0.0, 120.0, 120.0, 120.0, 120.0, 0.0, 120.0, 120.0, 0.0, 120.0, 120.0, 0.0, 120.0, 0.0, 120.0, 120.0, 0.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 0.0, 120.0, 0.0, 120.0, 120.0, 0.0, 676.0, 0.0, 20.0, 120.0, 120.0, 120.0, 120.0, 120.0, 100.0, 20.0, 0.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 0.0, 120.0, 120.0, 120.0, 120.0]"
4,0.826746,97,192.168.128.50,192.168.120.158,443,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 696.0, 120.0, 120.0, 120.0, 120.0, 120.0, ..."
5,0.822222,3501,192.168.1.220,192.168.1.255,137,"[1.0, 0.0, 1.0, 1.0, 1.0, 6.0, 0.0, 7.0, 0.0, 1.0, 0.0, 0.0, 1.0, 4.0, 0.0, 0.0, 1.0, 0.0, 2.0, 11.0, 3.0, 0.0, 0.0, 5.0, 6.0, 34.0, 0.0, 9.0, 2.0, 0.0, 1.0, 5.0, 2.0, 5.0, 0.0, 4.0, 4.0, 2.0, 3.0, 12.0, 1.0, 1.0, 0.0, 2.0, 0.0, 2.0, 40.0, 6.0, 0.0, 15.0, 5.0, 3.0, 0.0, 1.0, 2.0, 0.0, 3.0, 1.0, 1.0, 0.0, 11.0, 4.0, 1.0, 0.0, 12.0, 37.0, 0.0, 1.0, 1.0, 2.0, 0.0, 1.0, 3.0, 3.0, 5.0, 1.0, 3.0, 2.0, 4.0, 1.0, 1.0, 6.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 5.0, 1.0, 0.0, 6.0, 2.0, 33.0, 8.0, 0.0, 7.0, 0.0, 0.0, ...]"
6,0.812698,1193,192.168.1.220,224.0.0.251,5353,"[9.0, 1.0, 13.0, 8.0, 1.0, 65.0, 6.0, 3.0, 7.0, 2.0, 8.0, 2.0, 3.0, 20.0, 1.0, 7.0, 14.0, 30.0, 5.0, 12.0, 1.0, 1.0, 10.0, 6.0, 2.0, 1.0, 18.0, 2.0, 11.0, 32.0, 8.0, 1.0, 2.0, 4.0, 1.0, 1.0, 1.0, 14.0, 17.0, 9.0, 51.0, 2.0, 11.0, 6.0, 3.0, 3.0, 14.0, 1.0, 3.0, 2.0, 1.0, 1.0, 2.0, 8.0, 1.0, 1.0, 1.0, 47.0, 1.0, 3.0,